In [173]:
import matplotlib
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import confusion_matrix
import warnings #to remove the warnings
warnings.filterwarnings('ignore')

In [174]:
cols = ['ID', 'Diagnosis', 'radius', 'texture', 'perimeter', 'area', 'smoothness', 'compactness', 'concavity', 'concave_points', 'symmetry', 'fractal_dimension',
        'radius_SE', 'texture_SE', 'perimeter_SE', 'area_SE', 'smoothness_SE', 'compactness_SE', 'concavity_SE', 'concave_points_SE', 'symmetry_SE', 'fractal_dimension_SE',
        'radius_MAX', 'texture_MAX', 'perimeter_MAX', 'area_MAX', 'smoothness_MAX', 'compactness_MAX', 'concavity_MAX', 'concave_points_MAX', 'symmetry_MAX', 'fractal_dimension_MAX']
df = pd.read_csv('../../../../../Wisconsin_Database/wdbc.data', header=None)
df.columns = cols
df['Diagnosis'] = df['Diagnosis'].map(
                   {'M':True,'B':False})
print(df.shape)

(569, 32)


In [57]:
X = df[cols[2:]]
y = df[cols[1]]

SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED)
print('X_train: {}, X_test: {}'.format(str(X_train.shape), str(X_test.shape)))
print('Y_train: {}, Y_test: {}'.format(str(y_train.shape), str(y_test.shape)))
scaler = StandardScaler()
scaler.fit(X_train)
col_names=df.columns[2:]
scaled_df = pd.DataFrame(X_train, columns=col_names)

X_train: (455, 30), X_test: (114, 30)
Y_train: (455,), Y_test: (114,)


In [58]:
X_train_List = []
X_test_list = []
for i in range(len(X_train.columns)):
    X_train_List.append(pd.DataFrame(X_train[X_train.columns[i]]))
    X_test_list.append(pd.DataFrame(X_test[X_test.columns[i]]))
print(X_train)
print(X_train_List[i])

     radius  texture  perimeter    area  smoothness  compactness  concavity  \
68    9.029    17.33      58.79   250.5     0.10660      0.14130    0.31300   
181  21.090    26.57     142.70  1311.0     0.11410      0.28320    0.24870   
63    9.173    13.86      59.20   260.9     0.07721      0.08751    0.05988   
248  10.650    25.22      68.01   347.0     0.09657      0.07234    0.02379   
60   10.170    14.88      64.55   311.9     0.11340      0.08061    0.01084   
..      ...      ...        ...     ...         ...          ...        ...   
71    8.888    14.64      58.79   244.0     0.09783      0.15310    0.08606   
106  11.640    18.33      75.17   412.5     0.11420      0.10170    0.07070   
270  14.290    16.82      90.30   632.6     0.06429      0.02675    0.00725   
435  13.980    19.62      91.12   599.5     0.10600      0.11330    0.11260   
102  12.180    20.52      77.22   458.7     0.08013      0.04038    0.02383   

     concave_points  symmetry  fractal_dimension  .

# SVM Feature Ranking

In [171]:


sumAcc = 0
positionalValues = {}
numTrainedModels = 100


for iteration in range(numTrainedModels):
    X = df[cols[2:]]
    y = df[cols[1]]
    SEED = 1000
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED + iteration)
    scaler = StandardScaler()
    scaler.fit(X_train)
    col_names=df.columns[2:]
    scaled_df = pd.DataFrame(X_train, columns=col_names)

    '''Defining List of Columns that Can be Used'''

    X_train_List = []
    X_test_list = []
    for i in range(len(X_train.columns)):
        X_train_List.append(pd.DataFrame(X_train[X_train.columns[i]]))
        X_test_list.append(pd.DataFrame(X_test[X_test.columns[i]]))
    results = {}
    currentlyUsedTrainDataframe = pd.DataFrame()
    currentlyUsedTestDataframe = pd.DataFrame()
    colsInUse = []
    currentBest = 0

    for i in range(len(X_train_List)):
        tempResults = {}
        if i == 0:
            '''Identifies Best First Value'''
            for j in range(len(X_train_List)):

                svclassifier = SVC(kernel='linear')
                svclassifier.fit(pd.DataFrame(X_train_List[j]), y_train)
                y_pred = svclassifier.predict(X_test_list[j])
                performanceAccuracy = sum(y_pred == y_test)/len(y_test)
                tempResults[X_train_List[j].columns[0]] = performanceAccuracy
        else:
            '''Identifies Best Value for every value after first'''
            for j in range(len(X_train_List)):
                tempTrainList = currentlyUsedTrainDataframe.copy()
                tempTrainList[X_train_List[j].columns[0]] = X_train[X_train_List[j].columns[0]]
                tempTestList = currentlyUsedTestDataframe.copy()
                tempTestList[X_test_list[j].columns[0]] = X_test[X_test_list[j].columns[0]]
                svclassifier = SVC(kernel='linear')
                svclassifier.fit(pd.DataFrame(X_train_List[j]), y_train)
                y_pred = svclassifier.predict(X_test_list[j])
                performanceAccuracy = sum(y_pred == y_test)/len(y_test)
                tempResults[X_train_List[j].columns[0]] = performanceAccuracy

        '''Sorted all Additional feature results, identifying the best'''
        sort_dict = dict(sorted(tempResults.items(), key=lambda item: item[1])) 
        sort_dict = dict(reversed(sort_dict.items()))
        name = list(sort_dict.keys())[0]
        value = sort_dict[name]
        '''Adds Column to List of Cols in Use and results of that combination to final list'''
        colsInUse.append(name)
        results[str(colsInUse)] = value
        currentlyUsedTrainDataframe[name] = X_train[name]
        currentlyUsedTestDataframe[name] = X_test[name]
        '''Deletes Column from list of columns remaining'''
        for j in range(len(X_train_List)):
            if X_train_List[j].columns[0] == name:
                X_train_List.pop(j)
                X_test_list.pop(j)
                break

    sortedResults = dict(sorted(results.items(), key=lambda item: item[1])) 
    sortedResults = dict(reversed(sortedResults.items()))
    #print('{}: {}'.format(list(sortedResults.keys())[0], list(sortedResults.values())[0]))
    sumAcc += list(sortedResults.values())[0]
    '''Takes list as a string and returns as a list'''
    featureOrder = list(results.keys())[-1][1:-1].replace(' ', '').replace("'", '').split(',')
    for i in range(len(featureOrder)):
        if featureOrder[i] in positionalValues:
            positionalValues[featureOrder[i]] = np.hstack((positionalValues[featureOrder[i]], i))
        else:
            positionalValues[featureOrder[i]] = np.array(i)


#positionalValues = dict(sorted(positionalValues.items(), key=lambda item: item[1]))
averagePos = {}
minPos = {}
maxPos = {}
stdErrPos = {}
for i in positionalValues:
    averagePos[i] = np.mean(positionalValues[i])
    minPos[i] = min(positionalValues[i])
    maxPos[i] = max(positionalValues[i])
    stdErrPos[i] = np.std(positionalValues[i])
    np.mean(positionalValues[i])
sortedVals = dict(sorted(averagePos.items(), key=lambda item: item[1]))
for i in sortedVals:
    print('[{}] \t Mean Val: {}, Min Val: {}, Max Val: {}, StdErr: {}'.format(i, averagePos[i], minPos[i], maxPos[i], stdErrPos[i]))

[area_MAX] 	 Mean Val: 1.16, Min Val: 0, Max Val: 5, StdErr: 1.146472851837321
[perimeter_MAX] 	 Mean Val: 1.32, Min Val: 0, Max Val: 5, StdErr: 1.2480384609458155
[radius_MAX] 	 Mean Val: 1.6, Min Val: 0, Max Val: 5, StdErr: 1.0392304845413265
[area] 	 Mean Val: 4.28, Min Val: 0, Max Val: 8, StdErr: 1.5689486925964147
[concave_points_MAX] 	 Mean Val: 4.38, Min Val: 0, Max Val: 9, StdErr: 2.497118339206214
[perimeter] 	 Mean Val: 5.22, Min Val: 0, Max Val: 11, StdErr: 1.9213536894595957
[area_SE] 	 Mean Val: 5.87, Min Val: 0, Max Val: 10, StdErr: 2.0816099538578308
[radius] 	 Mean Val: 5.92, Min Val: 2, Max Val: 10, StdErr: 1.604244370412438
[concavity_MAX] 	 Mean Val: 7.32, Min Val: 0, Max Val: 11, StdErr: 2.1857721747702805
[concavity] 	 Mean Val: 9.16, Min Val: 3, Max Val: 12, StdErr: 1.6353592877407703
[radius_SE] 	 Mean Val: 10.41, Min Val: 4, Max Val: 14, StdErr: 1.4838800490605701
[perimeter_SE] 	 Mean Val: 10.62, Min Val: 7, Max Val: 14, StdErr: 1.4197182819137042
[compactness_

# Naive Bayes Feature Ranking

In [170]:
sumAcc = 0
positionalValues = {}
numTrainedModels = 100


for iteration in range(numTrainedModels):
    X = df[cols[2:]]
    y = df[cols[1]]
    SEED = 1000
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED + iteration)
    scaler = StandardScaler()
    scaler.fit(X_train)
    col_names=df.columns[2:]
    scaled_df = pd.DataFrame(X_train, columns=col_names)

    '''Defining List of Columns that Can be Used'''

    X_train_List = []
    X_test_list = []
    for i in range(len(X_train.columns)):
        X_train_List.append(pd.DataFrame(X_train[X_train.columns[i]]))
        X_test_list.append(pd.DataFrame(X_test[X_test.columns[i]]))
    results = {}
    currentlyUsedTrainDataframe = pd.DataFrame()
    currentlyUsedTestDataframe = pd.DataFrame()
    colsInUse = []
    currentBest = 0

    for i in range(len(X_train_List)):
        tempResults = {}
        if i == 0:
            '''Identifies Best First Value'''
            for j in range(len(X_train_List)):
                nb = GaussianNB()
                nb.fit(pd.DataFrame(X_train_List[j]), y_train)
                y_pred = (nb.predict(X_test_list[j]))
                performanceAccuracy = sum(y_pred == y_test)/len(y_test)
                tempResults[X_train_List[j].columns[0]] = performanceAccuracy
        else:
            '''Identifies Best Value for every value after first'''
            for j in range(len(X_train_List)):
                nb = GaussianNB()
                tempTrainList = currentlyUsedTrainDataframe.copy()
                tempTrainList[X_train_List[j].columns[0]] = X_train[X_train_List[j].columns[0]]
                tempTestList = currentlyUsedTestDataframe.copy()
                tempTestList[X_test_list[j].columns[0]] = X_test[X_test_list[j].columns[0]]
                nb.fit(pd.DataFrame(tempTrainList), y_train)
                y_pred = (nb.predict(tempTestList))
                performanceAccuracy = sum(y_pred == y_test)/len(y_test)
                tempResults[X_train_List[j].columns[0]] = performanceAccuracy

        '''Sorted all Additional feature results, identifying the best'''
        sort_dict = dict(sorted(tempResults.items(), key=lambda item: item[1])) 
        sort_dict = dict(reversed(sort_dict.items()))
        name = list(sort_dict.keys())[0]
        value = sort_dict[name]
        '''Adds Column to List of Cols in Use and results of that combination to final list'''
        colsInUse.append(name)
        results[str(colsInUse)] = value
        currentlyUsedTrainDataframe[name] = X_train[name]
        currentlyUsedTestDataframe[name] = X_test[name]
        '''Deletes Column from list of columns remaining'''
        for j in range(len(X_train_List)):
            if X_train_List[j].columns[0] == name:
                X_train_List.pop(j)
                X_test_list.pop(j)
                break

    sortedResults = dict(sorted(results.items(), key=lambda item: item[1])) 
    sortedResults = dict(reversed(sortedResults.items()))
    #print('{}: {}'.format(list(sortedResults.keys())[0], list(sortedResults.values())[0]))
    sumAcc += list(sortedResults.values())[0]
    '''Takes list as a string and returns as a list'''
    featureOrder = list(results.keys())[-1][1:-1].replace(' ', '').replace("'", '').split(',')
    for i in range(len(featureOrder)):
        if featureOrder[i] in positionalValues:
            positionalValues[featureOrder[i]] = np.hstack((positionalValues[featureOrder[i]], i))
        else:
            positionalValues[featureOrder[i]] = np.array(i)


#positionalValues = dict(sorted(positionalValues.items(), key=lambda item: item[1]))
averagePos = {}
minPos = {}
maxPos = {}
stdErrPos = {}
for i in positionalValues:
    averagePos[i] = np.mean(positionalValues[i])
    minPos[i] = min(positionalValues[i])
    maxPos[i] = max(positionalValues[i])
    stdErrPos[i] = np.std(positionalValues[i])
    np.mean(positionalValues[i])
sortedVals = dict(sorted(averagePos.items(), key=lambda item: item[1]))
for i in sortedVals:
    print('[{}] \t Mean Val: {}, Min Val: {}, Max Val: {}, StdErr: {}'.format(i, averagePos[i], minPos[i], maxPos[i], stdErrPos[i]))

[texture_MAX] 	 Mean Val: 5.53, Min Val: 1, Max Val: 21, StdErr: 4.9019485921417
[fractal_dimension_SE] 	 Mean Val: 5.65, Min Val: 1, Max Val: 15, StdErr: 2.783433131943356
[perimeter_MAX] 	 Mean Val: 6.38, Min Val: 0, Max Val: 23, StdErr: 7.216342563930845
[smoothness_MAX] 	 Mean Val: 6.48, Min Val: 1, Max Val: 19, StdErr: 5.172001546790178
[symmetry_SE] 	 Mean Val: 6.85, Min Val: 2, Max Val: 16, StdErr: 2.8926631328241457
[concave_points_MAX] 	 Mean Val: 6.96, Min Val: 0, Max Val: 25, StdErr: 7.822940623576277
[area_MAX] 	 Mean Val: 9.32, Min Val: 0, Max Val: 29, StdErr: 8.901550426751509
[concave_points_SE] 	 Mean Val: 9.34, Min Val: 2, Max Val: 21, StdErr: 4.210035629302915
[smoothness_SE] 	 Mean Val: 9.36, Min Val: 3, Max Val: 20, StdErr: 3.6180657816020974
[fractal_dimension_MAX] 	 Mean Val: 9.78, Min Val: 1, Max Val: 28, StdErr: 6.250727957606219
[symmetry_MAX] 	 Mean Val: 9.93, Min Val: 1, Max Val: 25, StdErr: 7.136182452824479
[concavity_SE] 	 Mean Val: 10.03, Min Val: 1, Max 

# KNN Feature Selection


In [184]:
sumAcc = 0
positionalValues = {}
numTrainedModels = 100


for iteration in range(numTrainedModels):
    X = df[cols[2:]]
    y = df[cols[1]]
    SEED = 1000
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED + iteration)
    scaler = StandardScaler()
    scaler.fit(X_train)
    col_names=df.columns[2:]
    scaled_df = pd.DataFrame(X_train, columns=col_names)

    '''Defining List of Columns that Can be Used'''

    X_train_List = []
    X_test_list = []
    for i in range(len(X_train.columns)):
        X_train_List.append(pd.DataFrame(X_train[X_train.columns[i]]))
        X_test_list.append(pd.DataFrame(X_test[X_test.columns[i]]))
    results = {}
    currentlyUsedTrainDataframe = pd.DataFrame()
    currentlyUsedTestDataframe = pd.DataFrame()
    colsInUse = []
    currentBest = 0

    for i in range(len(X_train_List)):
        tempResults = {}
        if i == 0:
            '''Identifies Best First Value'''
            for j in range(len(X_train_List)):
                error = []
                score_vals = []
                for neighbors in range(1, 17):
                    knn = KNeighborsRegressor(n_neighbors=neighbors)
                    knn.fit(pd.DataFrame(X_train_List[j]), y_train)
                    pred_i = knn.predict(X_test_list[j]).round(decimals=0)
                    mae = mean_absolute_error(y_test, pred_i)
                    error.append(mae)
                    score_vals.append(knn.score(X_test_list[j], y_test))
                bestIndex = score_vals.index(max(score_vals)) + 1  #Adds one because it starts with 1 neighbor not 0

                regressor = KNeighborsRegressor(n_neighbors=bestIndex)
                regressor.fit(pd.DataFrame(X_train_List[j]), y_train)
                y_pred = regressor.predict(X_test_list[j]).round(decimals=0)
                y_pred = y_pred == 1
                #print('Iteration {}: '.format(i))
                performanceAccuracy = sum(y_pred == y_test)/len(y_test)
                tempResults[X_train_List[j].columns[0]] = performanceAccuracy
        else:
            '''Identifies Best Value for every value after first'''
            for j in range(len(X_train_List)):
                tempTrainList = currentlyUsedTrainDataframe.copy()
                tempTrainList[X_train_List[j].columns[0]] = X_train[X_train_List[j].columns[0]]
                tempTestList = currentlyUsedTestDataframe.copy()
                tempTestList[X_test_list[j].columns[0]] = X_test[X_test_list[j].columns[0]]
                error = []
                score_vals = []
                for neighbors in range(1, 17):
                    knn = KNeighborsRegressor(n_neighbors=neighbors)
                    knn.fit(pd.DataFrame(tempTrainList), y_train)
                    pred_i = knn.predict(tempTestList).round(decimals=0)
                    mae = mean_absolute_error(y_test, pred_i)
                    error.append(mae)
                    score_vals.append(knn.score(tempTestList, y_test))
                bestIndex = score_vals.index(max(score_vals)) + 1  #Adds one because it starts with 1 neighbor not 0

                regressor = KNeighborsRegressor(n_neighbors=bestIndex)
                regressor.fit(pd.DataFrame(tempTrainList), y_train)
                y_pred = regressor.predict(tempTestList).round(decimals=0)
                y_pred = y_pred == 1
                performanceAccuracy = sum(y_pred == y_test)/len(y_test)
                tempResults[X_train_List[j].columns[0]] = performanceAccuracy

        '''Sorted all Additional feature results, identifying the best'''
        sort_dict = dict(sorted(tempResults.items(), key=lambda item: item[1])) 
        sort_dict = dict(reversed(sort_dict.items()))
        name = list(sort_dict.keys())[0]
        value = sort_dict[name]
        '''Adds Column to List of Cols in Use and results of that combination to final list'''
        colsInUse.append(name)
        results[str(colsInUse)] = value
        currentlyUsedTrainDataframe[name] = X_train[name]
        currentlyUsedTestDataframe[name] = X_test[name]
        '''Deletes Column from list of columns remaining'''
        for j in range(len(X_train_List)):
            if X_train_List[j].columns[0] == name:
                X_train_List.pop(j)
                X_test_list.pop(j)
                break

    sortedResults = dict(sorted(results.items(), key=lambda item: item[1])) 
    sortedResults = dict(reversed(sortedResults.items()))
    #print('{}: {}'.format(list(sortedResults.keys())[0], list(sortedResults.values())[0]))
    sumAcc += list(sortedResults.values())[0]
    '''Takes list as a string and returns as a list'''
    featureOrder = list(results.keys())[-1][1:-1].replace(' ', '').replace("'", '').split(',')
    for i in range(len(featureOrder)):
        if featureOrder[i] in positionalValues:
            positionalValues[featureOrder[i]] = np.hstack((positionalValues[featureOrder[i]], i))
        else:
            positionalValues[featureOrder[i]] = np.array(i)
    print(iteration)

#positionalValues = dict(sorted(positionalValues.items(), key=lambda item: item[1]))
averagePos = {}
minPos = {}
maxPos = {}
stdErrPos = {}
for i in positionalValues:
    averagePos[i] = np.mean(positionalValues[i])
    minPos[i] = min(positionalValues[i])
    maxPos[i] = max(positionalValues[i])
    stdErrPos[i] = np.std(positionalValues[i])
    np.mean(positionalValues[i])
sortedVals = dict(sorted(averagePos.items(), key=lambda item: item[1]))
for i in sortedVals:
    print('[{}] \t Mean Val: {}, Min Val: {}, Max Val: {}, StdErr: {}'.format(i, averagePos[i], minPos[i], maxPos[i], stdErrPos[i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[concave_points_MAX] 	 Mean Val: 3.36, Min Val: 0, Max Val: 15, StdErr: 3.863987577619783
[fractal_dimension_MAX] 	 Mean Val: 3.9, Min Val: 1, Max Val: 16, StdErr: 3.2357379374726873
[fractal_dimension_SE] 	 Mean Val: 6.5, Min Val: 1, Max Val: 13, StdErr: 2.99833287011299
[smoothness_MAX] 	 Mean Val: 7.58, Min Val: 1, Max Val: 17, StdErr: 3.592158125695471
[symmetry_MAX] 	 Mean Val: 8.01, Min Val: 1, Max Val: 18, StdErr: 5.233536089490547
[symmetry_SE] 	 Mean Val: 8.07, Min Val: 1, Max Val: 14, StdErr: 2.7212313389346376
[concave_points_SE] 	 Mean Val: 8.37, Min Val: 1, Max Val: 15, StdErr: 3.369436154611035
[concavity_MAX] 	 Mean Val: 9.86, Min Val: 1, Max Val: 23, StdErr: 5.964930846204338
[concavit

## Stopping After no Longer Improving

In [109]:
sumAcc = 0
positionalValues = {}

for iteration in range(100):
    X = df[cols[2:]]
    y = df[cols[1]]
    SEED = 42
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED + iteration)
    scaler = StandardScaler()
    scaler.fit(X_train)
    col_names=df.columns[2:]
    scaled_df = pd.DataFrame(X_train, columns=col_names)

    '''Defining List of Columns that Can be Used'''

    X_train_List = []
    X_test_list = []
    for i in range(len(X_train.columns)):
        X_train_List.append(pd.DataFrame(X_train[X_train.columns[i]]))
        X_test_list.append(pd.DataFrame(X_test[X_test.columns[i]]))
    results = {}
    currentlyUsedTrainDataframe = pd.DataFrame()
    currentlyUsedTestDataframe = pd.DataFrame()
    colsInUse = []
    currentBest = 0

    for i in range(len(X_train_List)):
        tempResults = {}
        if i == 0:
            '''Identifies Best First Value'''
            for j in range(len(X_train_List)):
                nb = GaussianNB()
                nb.fit(pd.DataFrame(X_train_List[j]), y_train)
                y_pred = (nb.predict(X_test_list[j]))
                performanceAccuracy = sum(y_pred == y_test)/len(y_test)
                tempResults[X_train_List[j].columns[0]] = performanceAccuracy
        else:
            '''Identifies Best Value for every value after first'''
            for j in range(len(X_train_List)):
                nb = GaussianNB()
                tempTrainList = currentlyUsedTrainDataframe.copy()
                tempTrainList[X_train_List[j].columns[0]] = X_train[X_train_List[j].columns[0]]
                tempTestList = currentlyUsedTestDataframe.copy()
                tempTestList[X_test_list[j].columns[0]] = X_test[X_test_list[j].columns[0]]
                nb.fit(pd.DataFrame(tempTrainList), y_train)
                y_pred = (nb.predict(tempTestList))
                performanceAccuracy = sum(y_pred == y_test)/len(y_test)
                tempResults[X_train_List[j].columns[0]] = performanceAccuracy

        '''Sorted all Additional feature results, identifying the best'''
        sort_dict = dict(sorted(tempResults.items(), key=lambda item: item[1])) 
        sort_dict = dict(reversed(sort_dict.items()))
        name = list(sort_dict.keys())[0]
        value = sort_dict[name]
        if value > currentBest:
            currentBest = value
        else:
            break

        '''Adds Column to List of Cols in Use and results of that combination to final list'''
        colsInUse.append(name)
        results[str(colsInUse)] = value
        currentlyUsedTrainDataframe[name] = X_train[name]
        currentlyUsedTestDataframe[name] = X_test[name]
        '''Deletes Column from list of columns remaining'''
        for j in range(len(X_train_List)):
            if X_train_List[j].columns[0] == name:
                X_train_List.pop(j)
                X_test_list.pop(j)
                break

    sortedResults = dict(sorted(results.items(), key=lambda item: item[1])) 
    sortedResults = dict(reversed(sortedResults.items()))
    print('{}: {}'.format(list(sortedResults.keys())[0], list(sortedResults.values())[0]))
    sumAcc += list(sortedResults.values())[0]
print(sumAcc/100)


['area_MAX', 'concavity_MAX', 'smoothness_MAX']: 0.9736842105263158
['perimeter_MAX', 'smoothness_MAX', 'texture_MAX']: 0.9912280701754386
['concave_points_MAX']: 0.9824561403508771
['perimeter_MAX', 'smoothness_MAX', 'texture_MAX']: 0.9736842105263158
['area_MAX', 'fractal_dimension_MAX', 'concave_points_MAX', 'texture_MAX']: 0.9824561403508771
['concave_points', 'texture_MAX']: 0.9649122807017544
['area_MAX', 'symmetry_MAX', 'concavity_MAX']: 0.956140350877193
['concave_points_MAX', 'perimeter_MAX', 'concavity_MAX']: 0.9736842105263158
['perimeter_MAX', 'smoothness_MAX']: 0.9824561403508771
['concave_points_MAX', 'smoothness_MAX', 'area_MAX', 'texture_MAX', 'concavity_MAX']: 0.956140350877193
['area_MAX', 'symmetry_MAX', 'concave_points_MAX']: 0.9912280701754386
['concave_points_MAX', 'area_MAX', 'fractal_dimension_MAX']: 0.9912280701754386
['concave_points_MAX', 'area_MAX']: 1.0
['area_MAX', 'perimeter_MAX']: 0.956140350877193
['concave_points', 'texture', 'concavity_SE', 'fractal_d